#. This notebook is for selecting subjects from ICUstays table, selecting chartevents features and their values from chartevents then merge

In [1]:
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings("ignore", category=Warning)

1. Extract the first ICU stay per patient: include patients with at least one ICU admission. If a patient has multiple ICU stays, we keep the first ICU admission.

In [2]:
icustays = pd.read_csv("../mimic-iv-3.1/icu/icustays.csv", parse_dates=["intime", "outtime"])
icustays

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000690,25860671,37081114,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252
2,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
3,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
4,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
...,...,...,...,...,...,...,...,...
94453,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
94454,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
94455,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
94456,19999840,21033226,38978960,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


In [3]:
icustays.info()
icustays.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94458 entries, 0 to 94457
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   subject_id      94458 non-null  int64         
 1   hadm_id         94458 non-null  int64         
 2   stay_id         94458 non-null  int64         
 3   first_careunit  94458 non-null  object        
 4   last_careunit   94458 non-null  object        
 5   intime          94458 non-null  datetime64[ns]
 6   outtime         94444 non-null  datetime64[ns]
 7   los             94444 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(3), object(2)
memory usage: 5.8+ MB


subject_id         0
hadm_id            0
stay_id            0
first_careunit     0
last_careunit      0
intime             0
outtime           14
los               14
dtype: int64

In [4]:
# there are subjects whose outtime is not known, remove them 
icustays = icustays.dropna(subset=['outtime', 'los'])
icustays.isnull().sum()
icustays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94444 entries, 0 to 94457
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   subject_id      94444 non-null  int64         
 1   hadm_id         94444 non-null  int64         
 2   stay_id         94444 non-null  int64         
 3   first_careunit  94444 non-null  object        
 4   last_careunit   94444 non-null  object        
 5   intime          94444 non-null  datetime64[ns]
 6   outtime         94444 non-null  datetime64[ns]
 7   los             94444 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(3), object(2)
memory usage: 6.5+ MB


In [5]:
# Sort by subject_id, hadm_id and ICU admission time
icustays_sorted = icustays.sort_values(by=["subject_id",'hadm_id', "intime"])
icustays_sorted

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000690,25860671,37081114,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252
2,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
3,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
4,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
...,...,...,...,...,...,...,...,...
94453,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
94454,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
94455,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
94456,19999840,21033226,38978960,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


In [6]:
# Keep only the first ICU stay per patient
first_stays = icustays_sorted.drop_duplicates(subset=["subject_id"], keep="first")
first_stays

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000690,25860671,37081114,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252
2,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
3,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
5,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
...,...,...,...,...,...,...,...,...
94453,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
94454,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
94455,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
94456,19999840,21033226,38978960,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


In [8]:
# Calculate duration in hours
first_stays["duration_hours"] = (first_stays["outtime"] - first_stays["intime"]).dt.total_seconds() / 3600 
desc = first_stays["duration_hours"].describe()
print(desc)
print(first_stays["duration_hours"].quantile([0.80, 0.85, 0.90, 0.95]))

count    65355.000000
mean        83.233193
std        120.421071
min          0.030000
25%         26.070833
50%         45.683611
75%         88.607639
max       3361.288056
Name: duration_hours, dtype: float64
0.80    104.904833
0.85    133.688583
0.90    181.362222
0.95    287.938000
Name: duration_hours, dtype: float64


=====>.     We filter ICU stays to those between 24 and 168 hours (1–7 days). This range excludes short stays that may lack sufficient physiological data and removes long-stay outliers (>7 days) that often involve complex, atypical clinical courses. While the IQR spans 26–89 hours, extending the upper limit to 168 hours ensures we capture the vast majority of clinically relevant ICU stays while maintaining a focus on early, acute care where predictive modeling is most effective.

In [9]:
# Filter between 12 and 168 hours
filtered_stays = first_stays[(first_stays["duration_hours"] >= 24) & (first_stays["duration_hours"] <= 168)].copy()
filtered_stays

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,duration_hours
1,10000690,25860671,37081114,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
3,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,26.832778
5,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,32.126111
8,10002013,23581541,39060235,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2160-05-18 10:00:53,2160-05-19 17:33:33,1.314352,31.544444
9,10002114,27793700,34672098,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2162-02-17 23:30:00,2162-02-20 21:16:27,2.907257,69.774167
...,...,...,...,...,...,...,...,...,...
94450,19999287,20175828,35165301,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2197-08-04 00:02:00,2197-08-08 16:58:17,4.705752,112.938056
94453,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,166.808889
94455,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,42.983889
94456,19999840,21033226,38978960,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766,127.146389


In [10]:
# select features which are needed to predict in hospital mortality 
# then select the values for the subject for the selected features

selected_feature_in_chart= [220179, 225624, 220615, 223835, 220045, 220645, 220210,
    223830, 220224, 220545, 220739, 223900, 223901, 223761,223762]

chunks = []

for chunk in pd.read_csv('../mimic-iv-3.1/icu/chartevents.csv', chunksize=10000): 
        filtered = chunk[chunk['itemid'].isin(selected_feature_in_chart)]
        chunks.append(filtered)

chart_events = pd.concat(chunks, ignore_index=True)
chart_events.head() 

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,18704.0,2180-07-23 14:00:00,2180-07-23 14:20:00,223761,98.7,98.7,°F,0.0
1,10000032,29079034,39553978,18704.0,2180-07-23 14:11:00,2180-07-23 14:17:00,220179,84,84.0,mmHg,0.0
2,10000032,29079034,39553978,18704.0,2180-07-23 14:12:00,2180-07-23 14:17:00,220045,91,91.0,bpm,0.0
3,10000032,29079034,39553978,18704.0,2180-07-23 14:12:00,2180-07-23 14:17:00,220210,24,24.0,insp/min,0.0
4,10000032,29079034,39553978,18704.0,2180-07-23 14:30:00,2180-07-23 14:43:00,220045,93,93.0,bpm,0.0


In [11]:
# the temp feature is in cel and fara, i am changing it to fara  
chart_events.loc[chart_events['itemid'] == 223762, 'valuenum'] = chart_events.loc[chart_events['itemid'] == 223762, 'valuenum'] * 9/5 + 32
chart_events.loc[chart_events['itemid'].isin([223761, 223762]), 'itemid'] = 223761 

chart_events['charttime'] = pd.to_datetime(chart_events['charttime'])

chart_events = chart_events.drop(['caregiver_id','storetime','warning'], axis=1).copy()
chart_events.nunique()

subject_id      65366
hadm_id         85242
stay_id         94445
charttime     5519894
itemid             14
value            2574
valuenum         2430
valueuom            9
dtype: int64

In [12]:
itemid_name_map = {
    220045: 'HeartRate',
    220739: 'GCS_Eye',
    223900: 'GCS_Verbal',
    223901: 'GCS_Motor',
    220210: 'RespiratoryRate',
    220645: 'Sodium',
    220615: 'Creatinine',
    225624: 'BUN',
    220545: 'Hematocrit',
    223761: 'TempF',
    220179: 'NIBP_Systolic',
    223835: 'FiO2',
    223830: 'pH_Arterial',
    220224: 'PaO2'}

itemid_list = [220045, 220739, 223900, 223901, 220210,220645, 220615, 225624, 220545, 223761, 220179,223835, 223830, 220224]

stats = []

for itemid in itemid_list:
    subset = chart_events[chart_events['itemid'] == itemid]['valuenum'].dropna()
    if subset.empty:
        continue
    stats.append({
        'itemid': itemid,
        'mean': subset.mean(),
        'std': subset.std(),
        'min': subset.min(),
        'p01': subset.quantile(0.01),
        'median': subset.median(),
        'p99': subset.quantile(0.99),
        'max': subset.max() })

features_distrubition = pd.DataFrame(stats)
#summary_df.to_csv("itemid_distribution_summary.csv", index=False)


features_distrubition

,itemid,mean,std,min,p01,median,p99,max
0,220045,88.003187,3801.504881,-241395.0,50.00,85.00,136.00,10000000.0
1,220739,3.281810,1.056274,1.0,1.00,4.00,4.00,4.0
2,223900,3.166343,1.865225,1.0,1.00,4.00,5.00,5.0
3,223901,5.248724,1.484820,1.0,1.00,6.00,6.00,6.0
4,220210,21.304515,2522.159697,0.0,8.00,19.00,37.00,7000400.0
5,220645,198.035917,7679.623229,-4.0,125.00,139.00,156.00,999999.0
6,220615,74.341385,8530.474402,-0.1,0.30,1.00,7.70,999999.0
7,225624,161.057559,11353.811830,0.0,4.00,24.00,126.00,999999.0
8,220545,176.085166,12246.515901,0.0,19.50,28.60,45.60,2011200.0
9,223761,98.691171,150.148221,-99.9,95.36,98.40,102.20,234123.0


In [13]:
item_caps = {
    223761: (95.0, 105.0),      # TempF (normal 97–100)
    220179: (70.0, 200.0),      # NIBP Systolic
    220045: (40.0, 200.0),      # HeartRate
    220210: (5.0, 60.0),        # RespiratoryRate
    220739: (1.0, 4.0),         # GCS_Eye
    223901: (1.0, 6.0),         # GCS_Motor
    223900: (1.0, 5.0),         # GCS_Verbal
    220615: (0.2, 10.0),        # Creatinine 
    220645: (120.0, 160.0),     # Sodium 
    225624: (4.0, 130.0),       # BUN 
    223835: (10.0, 100.0),      # FiO2 
    220224: (30.0, 500.0),      # PaO2 
    223830: (6.8, 7.6),         # pH_Arterial
    220545: (15.0, 55.0)}       # Hematocrit 

def clip_item_value(row, item_caps):
    itemid = row['itemid']
    val = row['valuenum']
    if pd.isnull(val):
        return np.nan
    if itemid in item_caps:
        lower, upper = item_caps[itemid]
        return np.clip(val, lower, upper)
    return val

chart_events['valuenum_capped'] = chart_events.apply(lambda row: clip_item_value(row, item_caps), axis=1)
chart_events

,subject_id,hadm_id,stay_id,charttime,itemid,value,valuenum,valueuom,valuenum_capped
0,10000032,29079034,39553978,2180-07-23 14:00:00,223761,98.7,98.70,°F,98.70
1,10000032,29079034,39553978,2180-07-23 14:11:00,220179,84,84.00,mmHg,84.00
2,10000032,29079034,39553978,2180-07-23 14:12:00,220045,91,91.00,bpm,91.00
3,10000032,29079034,39553978,2180-07-23 14:12:00,220210,24,24.00,insp/min,24.00
4,10000032,29079034,39553978,2180-07-23 14:30:00,220045,93,93.00,bpm,93.00
...,...,...,...,...,...,...,...,...,...
36229751,19999987,23865745,36195440,2145-11-04 05:01:00,220645,147,147.00,mEq/L,147.00
36229752,19999987,23865745,36195440,2145-11-04 05:01:00,225624,22,22.00,mg/dL,22.00
36229753,19999987,23865745,36195440,2145-11-04 07:18:00,220224,69,69.00,mmHg,69.00
36229754,19999987,23865745,36195440,2145-11-04 07:18:00,223830,7.42,7.42,units,7.42


In [14]:
chart_events.to_csv("../data/1_selected_chartevents_values.csv", index=False)

In [15]:
# Keep only values for ICU stays we're analyzing first ICU stay, between 24hrs and 7 days, then i will select for 48 hrs observation
selected_subject_merged_chartevents = chart_events.merge(
    filtered_stays,
    on=['subject_id', 'hadm_id','stay_id'],
    how='inner')

selected_subject_merged_chartevents

,subject_id,hadm_id,stay_id,charttime,itemid,value,valuenum,valueuom,valuenum_capped,first_careunit,last_careunit,intime,outtime,los,duration_hours
0,10000690,25860671,37081114,2150-11-04 12:00:00,223835,50,50.00,NaN,50.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
1,10000690,25860671,37081114,2150-11-04 18:00:00,223835,70,70.00,NaN,70.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
2,10000690,25860671,37081114,2150-11-06 08:00:00,220045,69,69.00,bpm,69.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
3,10000690,25860671,37081114,2150-11-06 08:00:00,220179,131,131.00,mmHg,131.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
4,10000690,25860671,37081114,2150-11-06 08:00:00,220210,23,23.00,insp/min,23.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12154938,19999987,23865745,36195440,2145-11-04 05:01:00,220645,147,147.00,mEq/L,147.00,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,46.508333
12154939,19999987,23865745,36195440,2145-11-04 05:01:00,225624,22,22.00,mg/dL,22.00,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,46.508333
12154940,19999987,23865745,36195440,2145-11-04 07:18:00,220224,69,69.00,mmHg,69.00,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,46.508333
12154941,19999987,23865745,36195440,2145-11-04 07:18:00,223830,7.42,7.42,units,7.42,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,46.508333


In [16]:
selected_subject_merged_chartevents.nunique()

subject_id           44421
hadm_id              44421
stay_id              44421
charttime          2686153
itemid                  14
value                 1739
valuenum              1877
valueuom                 9
valuenum_capped       1356
first_careunit          16
last_careunit           16
intime               44419
outtime              44421
los                  40998
duration_hours       40998
dtype: int64

In [17]:
selected_subject_merged_chartevents["charttime"] = pd.to_datetime(selected_subject_merged_chartevents["charttime"])
selected_subject_merged_chartevents["intime"] = pd.to_datetime(selected_subject_merged_chartevents["intime"])

selected_subject_merged_chartevents["hours_since_intime"] = (selected_subject_merged_chartevents["charttime"] -
                                                            selected_subject_merged_chartevents["intime"]).dt.total_seconds() / 3600

subject_with_48hrs_obs = selected_subject_merged_chartevents[selected_subject_merged_chartevents["hours_since_intime"] <= 48] 
subject_with_48hrs_obs.drop(columns='hours_since_intime', inplace=True)
subject_with_48hrs_obs

,subject_id,hadm_id,stay_id,charttime,itemid,value,valuenum,valueuom,valuenum_capped,first_careunit,last_careunit,intime,outtime,los,duration_hours
0,10000690,25860671,37081114,2150-11-04 12:00:00,223835,50,50.00,NaN,50.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
1,10000690,25860671,37081114,2150-11-04 18:00:00,223835,70,70.00,NaN,70.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
45,10000690,25860671,37081114,2150-11-02 19:54:00,220045,79,79.00,bpm,79.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
46,10000690,25860671,37081114,2150-11-02 19:54:00,220179,107,107.00,mmHg,107.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
47,10000690,25860671,37081114,2150-11-02 19:54:00,220210,23,23.00,insp/min,23.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12154938,19999987,23865745,36195440,2145-11-04 05:01:00,220645,147,147.00,mEq/L,147.00,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,46.508333
12154939,19999987,23865745,36195440,2145-11-04 05:01:00,225624,22,22.00,mg/dL,22.00,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,46.508333
12154940,19999987,23865745,36195440,2145-11-04 07:18:00,220224,69,69.00,mmHg,69.00,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,46.508333
12154941,19999987,23865745,36195440,2145-11-04 07:18:00,223830,7.42,7.42,units,7.42,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,46.508333


In [18]:
subject_with_48hrs_obs.nunique()

subject_id           44421
hadm_id              44421
stay_id              44421
charttime          2158763
itemid                  14
value                 1658
valuenum              1790
valueuom                 9
valuenum_capped       1317
first_careunit          16
last_careunit           16
intime               44419
outtime              44421
los                  40998
duration_hours       40998
dtype: int64

In [19]:
# csv file final cohort and their value 
subject_with_48hrs_obs.to_csv("../data/2_subject_with_48hrs_obs.csv", index=False)